In [ ]:
using Pkg; Pkg.activate("../../");
using Dates, Fastback

In [ ]:
# synthetic data
N = 100_000
prices = 10000.0 .+ cumsum(randn(N) .+ 0.1)
bids = prices .- 0.01
asks = prices .+ 0.01
dts = map(x -> DateTime(2000, 1, 1) + Minute(x) + Millisecond(123), 1:N);

### Long --> Short

In [ ]:
# create instrument
inst = Instrument(1, "AAPL")
insts = [inst]

# market data (order books)
data = MarketData(insts)

# create trading account
acc = Account(insts, 100_000.0)

show(acc)

pos = acc.positions[inst.index]

# backtest random trading strategy
for i in 1:N
    dt = dts[i]
    book = data.order_books[inst.index]
    update_book!(book, BidAsk(dts[i], bids[i], asks[i]))

    if i == 2
        execute_order!(acc, book, Order(inst, 1.0, dt))
        show(acc)
    end

    if i == N
        # close position
        execute_order!(acc, book, Order(inst, -1.0, dt))
        show(acc)
    end

    update_account!(acc, data, inst)
end

show(acc)

println("Sum realized pnl: $(sum(o.execution.realized_pnl for o in acc.orders_history))")
println("Net account pnl:  $(acc.equity - acc.initial_balance)")

### Short --> Long

In [ ]:
# create instrument
inst = Instrument(1, "AAPL")
insts = [inst]

# market data (order books)
data = MarketData(insts)

# create trading account
acc = Account(insts, 100_000.0)

pos = acc.positions[inst.index]

# backtest random trading strategy
for i in 1:N
    dt = dts[i]
    book = data.order_books[inst.index]
    update_book!(book, BidAsk(dts[i], bids[i], asks[i]))

    if i == 2
        execute_order!(acc, book, Order(inst, -1.0, dt))
        # println(pos.pnl)
        show(acc)
    end

    if i == N
        # close position
        execute_order!(acc, book, Order(inst, 1.0, dt))
        show(acc)
    end

    update_account!(acc, data, inst)
end

show(acc)

println("Sum realized pnl: $(sum(o.execution.realized_pnl for o in acc.orders_history))")
println("Net account pnl:  $(acc.equity - acc.initial_balance)")